In [14]:
import pandas as pd
import numpy as np

In [15]:
df = pd.read_csv("preprocess.csv");

In [16]:
df.head()

,Unnamed: 0,Position,Streams,Date,Region,Inv_Pos
0,21200,1,21030,2017-04-17,ec,199
1,21400,1,28054,2017-04-18,ec,199
2,21600,1,26153,2017-04-19,ec,199
3,21800,1,24244,2017-04-20,ec,199
4,22000,1,24873,2017-04-21,ec,199


In [17]:
df_countries = pd.read_json("countries.json")
df_countries = df_countries.transpose()
df_countries = df_countries.drop(columns=['continent', 'capital', 'languages', 'native', 'phone', 'currency'])
df_countries = df_countries.rename(columns={'name':'Country'})
df_countries.head()
df['Region'] = df['Region'].str.upper()
df = df.merge(df_countries, how='inner', left_on='Region', right_index=True)

In [18]:
df['Date'] = pd.to_datetime(df['Date'])
r = pd.date_range(start=df.Date.min(), end=df.Date.max())
r

DatetimeIndex(['2017-04-17', '2017-04-18', '2017-04-19', '2017-04-20',
               '2017-04-21', '2017-04-22', '2017-04-23', '2017-04-24',
               '2017-04-25', '2017-04-26',
               ...
               '2019-02-21', '2019-02-22', '2019-02-23', '2019-02-24',
               '2019-02-25', '2019-02-26', '2019-02-27', '2019-02-28',
               '2019-03-01', '2019-03-02'],
              dtype='datetime64[ns]', length=685, freq='D')

In [19]:
countries = [x for _, x in df.groupby('Country')]
countries

[       Unnamed: 0  Position  Streams       Date Region  Inv_Pos    Country
 414        169574         1   259697 2017-04-17     AR      199  Argentina
 415        169774         1   408219 2017-04-18     AR      199  Argentina
 416        169974         1   360378 2017-04-19     AR      199  Argentina
 417        170174         1   331557 2017-04-20     AR      199  Argentina
 418        170374         1   348161 2017-04-21     AR      199  Argentina
 419        170574         1   338829 2017-04-22     AR      199  Argentina
 420        170774         1   279433 2017-04-23     AR      199  Argentina
 421        170974         1   249803 2017-04-24     AR      199  Argentina
 422        171174         1   238474 2017-04-25     AR      199  Argentina
 423        171374         1   230988 2017-04-26     AR      199  Argentina
 424        171574         1   227884 2017-04-27     AR      199  Argentina
 425        171774         1   251385 2017-04-28     AR      199  Argentina
 426        

In [20]:
new_df = []
for country in countries:
    processed = country.set_index('Date').reindex(r, fill_value=country['Country'].values[0]).rename_axis('Date').reset_index()
    processed.loc[processed['Streams'] == processed['Country'], ['Streams']] = np.NaN
    processed['Streams'] = pd.to_numeric(processed['Streams'])
    processed.info()
    processed.index = pd.DatetimeIndex(processed.index)
    processed.fillna(0, inplace=True)
    new_df.append(processed)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 685 entries, 0 to 684
Data columns (total 7 columns):
Date          685 non-null datetime64[ns]
Unnamed: 0    685 non-null object
Position      685 non-null object
Streams       292 non-null float64
Region        685 non-null object
Inv_Pos       685 non-null object
Country       685 non-null object
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 37.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 685 entries, 0 to 684
Data columns (total 7 columns):
Date          685 non-null datetime64[ns]
Unnamed: 0    685 non-null object
Position      685 non-null object
Streams       442 non-null float64
Region        685 non-null object
Inv_Pos       685 non-null object
Country       685 non-null object
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 37.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 685 entries, 0 to 684
Data columns (total 7 columns):
Date          685 non-null datetime64[ns]
Unnamed:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 685 entries, 0 to 684
Data columns (total 7 columns):
Date          685 non-null datetime64[ns]
Unnamed: 0    685 non-null object
Position      685 non-null object
Streams       195 non-null float64
Region        685 non-null object
Inv_Pos       685 non-null object
Country       685 non-null object
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 37.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 685 entries, 0 to 684
Data columns (total 7 columns):
Date          685 non-null datetime64[ns]
Unnamed: 0    685 non-null object
Position      685 non-null object
Streams       159 non-null float64
Region        685 non-null object
Inv_Pos       685 non-null object
Country       685 non-null object
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 37.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 685 entries, 0 to 684
Data columns (total 7 columns):
Date          685 non-null datetime64[ns]
Unnamed:

In [21]:
new_df = pd.concat(new_df, axis=0)
new_df

,Date,Unnamed: 0,Position,Streams,Region,Inv_Pos,Country
1970-01-01 00:00:00.000000000,2017-04-17,169574,1,259697.0,AR,199,Argentina
1970-01-01 00:00:00.000000001,2017-04-18,169774,1,408219.0,AR,199,Argentina
1970-01-01 00:00:00.000000002,2017-04-19,169974,1,360378.0,AR,199,Argentina
1970-01-01 00:00:00.000000003,2017-04-20,170174,1,331557.0,AR,199,Argentina
1970-01-01 00:00:00.000000004,2017-04-21,170374,1,348161.0,AR,199,Argentina
1970-01-01 00:00:00.000000005,2017-04-22,170574,1,338829.0,AR,199,Argentina
1970-01-01 00:00:00.000000006,2017-04-23,170774,1,279433.0,AR,199,Argentina
1970-01-01 00:00:00.000000007,2017-04-24,170974,1,249803.0,AR,199,Argentina
1970-01-01 00:00:00.000000008,2017-04-25,171174,1,238474.0,AR,199,Argentina
1970-01-01 00:00:00.000000009,2017-04-26,171374,1,230988.0,AR,199,Argentina


In [22]:
new_df.to_csv("interpolated_rank.csv")

In [23]:
new_df.reset_index()
final_df = new_df.drop(['Unnamed: 0', 'Position', "Inv_Pos"], axis=1)
final_df.reset_index().to_csv("interpolated_rank_stripped.csv")

In [24]:
from sklearn import preprocessing

x = final_df.Streams.values #returns a numpy array
x = x.reshape(-1,1)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
final_df['Streams_norm'] = x_scaled

In [25]:
final_df.reset_index().drop(['index', 'Streams'],axis=1).to_csv("interpolated_rank_stripped.csv", index=False)